This Python 3 environment comes with many helpful analytics libraries installed
It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

Input data files are available in the read-only "../input/" directory
For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import os
import numpy as np
from PIL import Image
from torch.utils.data import Dataset
import torch
import pandas as pd
from torchvision import transforms
from torch.utils.data import random_split, DataLoader
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim

In [2]:
class ImageDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None, is_test=False):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.is_test = is_test

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if not self.is_test:
            # Fetch training images using paths from train.csv
            img_path = os.path.join(self.root_dir, self.data.iloc[idx, 1])
            image = Image.open(img_path).convert('RGB')
            label = int(self.data.iloc[idx, 0].split('_')[1]) - 1  # Labeling
            label = torch.tensor(label)
            if self.transform:
                image = self.transform(image)
            return image, label
        else:
            # For test data
            img_path = os.path.join(self.root_dir, self.data.iloc[idx, 1])
            image = Image.open(img_path).convert('RGB')
            if self.transform:
                image = self.transform(image)
            return image, self.data.iloc[idx, 0]

# Densenet169_ver5

In [3]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Create datasets
train_dataset = ImageDataset(
    csv_file='/kaggle/input/dl-63-cw-image-classification/train.csv',  # Correct path to train.csv
    root_dir='/kaggle/input/dl-63-cw-image-classification/train',  # Directory containing images
    transform=transform
)

test_dataset = ImageDataset(
    csv_file='/kaggle/input/dl-63-cw-image-classification/test.csv',  # Correct path to test.csv
    root_dir='/kaggle/input/dl-63-cw-image-classification/test/',  # Directory containing test images
    transform=transform, is_test=True
)

In [5]:
train_size = int(0.8 * len(train_dataset))
valid_size = len(train_dataset) - train_size

trainset, validset = random_split(train_dataset, [train_size, valid_size])

# Create DataLoaders
batch_size = 32
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(validset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [6]:
class CustomDenseNet169(nn.Module):
    def __init__(self, num_classes=71):
        super(CustomDenseNet169, self).__init__()
        
        # Load pre-trained DenseNet169 model
        self.base_model = models.densenet169(pretrained=True)
        
        # Freeze earlier layers if needed (optional)
        for param in self.base_model.features.parameters():
            param.requires_grad = False
        
        # Replace the classifier layer with a new fully connected layer
        self.base_model.classifier = nn.Sequential(
            nn.Linear(self.base_model.classifier.in_features, 256),  # in_features based on DenseNet architecture
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)  # Final output layer with num_classes outputs
        )

    def forward(self, x):
        return self.base_model(x)

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CustomDenseNet169(num_classes=71).to(device)

optimizer = optim.RMSprop(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)
criterion = nn.CrossEntropyLoss()

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet169_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet169_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet169-b2777c0a.pth" to /root/.cache/torch/hub/checkpoints/densenet169-b2777c0a.pth
100%|██████████| 54.7M/54.7M [00:00<00:00, 124MB/s] 


In [8]:
def train_and_val_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=10):
    best_val_acc = 0.0
    for epoch in range(num_epochs):
        model.train()  # Set to training mode
        running_loss, running_corrects = 0.0, 0

        # Training loop
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            _, preds = torch.max(outputs, 1)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = running_corrects.double() / len(train_loader.dataset)
        print(f'Training - Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}')

        # Validation loop
        model.eval()  # Set to evaluation mode
        val_corrects = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                val_corrects += torch.sum(preds == labels.data)

        val_acc = val_corrects.double() / len(val_loader.dataset)
        print(f'Validation Accuracy: {val_acc:.4f}')

        # Save the best model based on validation accuracy
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'best_model.pth')

In [9]:
train_and_val_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=15)

Training - Epoch 1/15, Loss: 2.2989, Accuracy: 0.4177
Validation Accuracy: 0.8358
Training - Epoch 2/15, Loss: 1.0369, Accuracy: 0.6995
Validation Accuracy: 0.8805
Training - Epoch 3/15, Loss: 0.7770, Accuracy: 0.7642
Validation Accuracy: 0.8931
Training - Epoch 4/15, Loss: 0.6390, Accuracy: 0.8005
Validation Accuracy: 0.9013
Training - Epoch 5/15, Loss: 0.5817, Accuracy: 0.8148
Validation Accuracy: 0.9019
Training - Epoch 6/15, Loss: 0.5174, Accuracy: 0.8397
Validation Accuracy: 0.9019
Training - Epoch 7/15, Loss: 0.4823, Accuracy: 0.8455
Validation Accuracy: 0.9063
Training - Epoch 8/15, Loss: 0.4511, Accuracy: 0.8598
Validation Accuracy: 0.9025
Training - Epoch 9/15, Loss: 0.4275, Accuracy: 0.8642
Validation Accuracy: 0.8906
Training - Epoch 10/15, Loss: 0.4063, Accuracy: 0.8683
Validation Accuracy: 0.9075
Training - Epoch 11/15, Loss: 0.3789, Accuracy: 0.8767
Validation Accuracy: 0.9013
Training - Epoch 12/15, Loss: 0.3632, Accuracy: 0.8814
Validation Accuracy: 0.9214
Training - Ep

In [11]:
for param in model.base_model.features[:6].parameters():
    param.requires_grad = False

# Re-define optimizer for fine-tuning
optimizer = optim.SGD(model.parameters(), lr=0.001)

# Fine-tune the model
train_and_val_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=15)

Training - Epoch 1/15, Loss: 0.3054, Accuracy: 0.8980
Validation Accuracy: 0.9126
Training - Epoch 2/15, Loss: 0.2657, Accuracy: 0.9119
Validation Accuracy: 0.9157
Training - Epoch 3/15, Loss: 0.2724, Accuracy: 0.9078
Validation Accuracy: 0.9138
Training - Epoch 4/15, Loss: 0.2582, Accuracy: 0.9183
Validation Accuracy: 0.9176
Training - Epoch 5/15, Loss: 0.2588, Accuracy: 0.9155
Validation Accuracy: 0.9189
Training - Epoch 6/15, Loss: 0.2509, Accuracy: 0.9149
Validation Accuracy: 0.9176
Training - Epoch 7/15, Loss: 0.2447, Accuracy: 0.9213
Validation Accuracy: 0.9151
Training - Epoch 8/15, Loss: 0.2411, Accuracy: 0.9228
Validation Accuracy: 0.9145
Training - Epoch 9/15, Loss: 0.2484, Accuracy: 0.9150
Validation Accuracy: 0.9164
Training - Epoch 10/15, Loss: 0.2371, Accuracy: 0.9235
Validation Accuracy: 0.9170
Training - Epoch 11/15, Loss: 0.2457, Accuracy: 0.9202
Validation Accuracy: 0.9157
Training - Epoch 12/15, Loss: 0.2396, Accuracy: 0.9199
Validation Accuracy: 0.9201
Training - Ep

In [12]:
def evaluate_model_on_test_set(model, test_loader, device, class_mapping):
    model.eval()
    predictions = []
    ids = []
    
    with torch.no_grad():
        for images, image_ids in test_loader:
            images = images.to(device)
            outputs = model(images)
            _, predicted_classes = torch.max(outputs, 1)
            
            predicted_classes = [class_mapping[p.item()] for p in predicted_classes]
            
        
            predictions.extend(predicted_classes)
            ids.extend(image_ids)  # image_ids chính là các ID từ file test.csv
    
    return ids, predictions

class_mapping = {i: f'class_{i+1}' for i in range(71)}

In [13]:
ids, predictions = evaluate_model_on_test_set(model, test_loader, device, class_mapping)
renamed_ids = list(range(len(ids)))
results = pd.DataFrame({
    'ID': renamed_ids, 
    'TARGET': predictions 
})

results.to_csv('submission9.csv', index=False)

# VER 6

In [4]:
train_size = int(0.8 * len(train_dataset))
valid_size = len(train_dataset) - train_size
trainset, validset = random_split(train_dataset, [train_size, valid_size])

batch_size = 64
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(validset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [5]:
class CustomDenseNet169(nn.Module):
    def __init__(self, num_classes=71):
        super(CustomDenseNet169, self).__init__()
        
        # Load pre-trained DenseNet169 model
        self.base_model = models.densenet169(pretrained=True)
        
        # Freeze earlier layers if needed (optional)
        for param in self.base_model.features.parameters():
            param.requires_grad = False
        
        # Replace the classifier layer with a new fully connected layer
        self.base_model.classifier = nn.Sequential(
            nn.Linear(self.base_model.classifier.in_features, 256),  # in_features based on DenseNet architecture
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)  # Final output layer with num_classes outputs
        )

    def forward(self, x):
        return self.base_model(x)

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CustomDenseNet169(num_classes=71).to(device)

optimizer = optim.RMSprop(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)
criterion = nn.CrossEntropyLoss()

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet169_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet169_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet169-b2777c0a.pth" to /root/.cache/torch/hub/checkpoints/densenet169-b2777c0a.pth
100%|██████████| 54.7M/54.7M [00:00<00:00, 183MB/s]


In [7]:
def train_and_val_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=10):
    best_val_acc = 0.0
    for epoch in range(num_epochs):
        model.train()  # Set to training mode
        running_loss, running_corrects = 0.0, 0

        # Training loop
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            _, preds = torch.max(outputs, 1)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = running_corrects.double() / len(train_loader.dataset)
        print(f'Training - Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}')

        # Validation loop
        model.eval()  # Set to evaluation mode
        val_corrects = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                val_corrects += torch.sum(preds == labels.data)

        val_acc = val_corrects.double() / len(val_loader.dataset)
        print(f'Validation Accuracy: {val_acc:.4f}')

        # Save the best model based on validation accuracy
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'best_model.pth')

In [8]:
train_and_val_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=10)

Training - Epoch 1/10, Loss: 2.3712, Accuracy: 0.4273
Validation Accuracy: 0.7862
Training - Epoch 2/10, Loss: 0.9902, Accuracy: 0.7196
Validation Accuracy: 0.8629
Training - Epoch 3/10, Loss: 0.7304, Accuracy: 0.7885
Validation Accuracy: 0.8899
Training - Epoch 4/10, Loss: 0.5983, Accuracy: 0.8255
Validation Accuracy: 0.8805
Training - Epoch 5/10, Loss: 0.5230, Accuracy: 0.8435
Validation Accuracy: 0.8912
Training - Epoch 6/10, Loss: 0.4571, Accuracy: 0.8576
Validation Accuracy: 0.9069
Training - Epoch 7/10, Loss: 0.4357, Accuracy: 0.8617
Validation Accuracy: 0.9019
Training - Epoch 8/10, Loss: 0.3962, Accuracy: 0.8757
Validation Accuracy: 0.9013
Training - Epoch 9/10, Loss: 0.3828, Accuracy: 0.8826
Validation Accuracy: 0.9094
Training - Epoch 10/10, Loss: 0.3453, Accuracy: 0.8888
Validation Accuracy: 0.9063


In [9]:
for param in model.base_model.features[:6].parameters():
    param.requires_grad = False

# Re-define optimizer for fine-tuning
optimizer = optim.SGD(model.parameters(), lr=0.001)

# Fine-tune the model
train_and_val_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=15)

Training - Epoch 1/15, Loss: 0.3383, Accuracy: 0.8921
Validation Accuracy: 0.9101
Training - Epoch 2/15, Loss: 0.3280, Accuracy: 0.8993
Validation Accuracy: 0.9201
Training - Epoch 3/15, Loss: 0.3140, Accuracy: 0.8998
Validation Accuracy: 0.9157
Training - Epoch 4/15, Loss: 0.3021, Accuracy: 0.9061
Validation Accuracy: 0.9176
Training - Epoch 5/15, Loss: 0.2972, Accuracy: 0.9084
Validation Accuracy: 0.9220
Training - Epoch 6/15, Loss: 0.2949, Accuracy: 0.9124
Validation Accuracy: 0.9233
Training - Epoch 7/15, Loss: 0.2946, Accuracy: 0.9092
Validation Accuracy: 0.9208
Training - Epoch 8/15, Loss: 0.2957, Accuracy: 0.9138
Validation Accuracy: 0.9189
Training - Epoch 9/15, Loss: 0.2842, Accuracy: 0.9138
Validation Accuracy: 0.9220
Training - Epoch 10/15, Loss: 0.2999, Accuracy: 0.9097
Validation Accuracy: 0.9233
Training - Epoch 11/15, Loss: 0.2858, Accuracy: 0.9144
Validation Accuracy: 0.9214
Training - Epoch 12/15, Loss: 0.2844, Accuracy: 0.9157
Validation Accuracy: 0.9226
Training - Ep

In [10]:
def evaluate_model_on_test_set(model, test_loader, device, class_mapping):
    model.eval()
    predictions = []
    ids = []
    
    with torch.no_grad():
        for images, image_ids in test_loader:
            images = images.to(device)
            outputs = model(images)
            _, predicted_classes = torch.max(outputs, 1)
            
            predicted_classes = [class_mapping[p.item()] for p in predicted_classes]
            
        
            predictions.extend(predicted_classes)
            ids.extend(image_ids)  # image_ids chính là các ID từ file test.csv
    
    return ids, predictions

class_mapping = {i: f'class_{i+1}' for i in range(71)}

In [11]:
ids, predictions = evaluate_model_on_test_set(model, test_loader, device, class_mapping)
renamed_ids = list(range(len(ids)))
results = pd.DataFrame({
    'ID': renamed_ids, 
    'TARGET': predictions 
})

results.to_csv('submission10.csv', index=False)

# Ver 7

In [12]:
train_size = int(0.8 * len(train_dataset))
valid_size = len(train_dataset) - train_size
trainset, validset = random_split(train_dataset, [train_size, valid_size])

batch_size = 32
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(validset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [13]:
class CustomDenseNet169(nn.Module):
    def __init__(self, num_classes=71):
        super(CustomDenseNet169, self).__init__()
        
        # Load pre-trained DenseNet169 model
        self.base_model = models.densenet169(pretrained=True)
        
        # Freeze earlier layers if needed (optional)
        for param in self.base_model.features.parameters():
            param.requires_grad = False
        
        # Replace the classifier layer with a new fully connected layer
        self.base_model.classifier = nn.Sequential(
            nn.Linear(self.base_model.classifier.in_features, 256),  # in_features based on DenseNet architecture
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)  # Final output layer with num_classes outputs
        )

    def forward(self, x):
        return self.base_model(x)

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CustomDenseNet169(num_classes=71).to(device)

optimizer = optim.RMSprop(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [15]:
def train_and_val_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=10):
    best_val_acc = 0.0
    for epoch in range(num_epochs):
        model.train()  # Set to training mode
        running_loss, running_corrects = 0.0, 0

        # Training loop
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            _, preds = torch.max(outputs, 1)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = running_corrects.double() / len(train_loader.dataset)
        print(f'Training - Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}')

        # Validation loop
        model.eval()  # Set to evaluation mode
        val_corrects = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                val_corrects += torch.sum(preds == labels.data)

        val_acc = val_corrects.double() / len(val_loader.dataset)
        print(f'Validation Accuracy: {val_acc:.4f}')

        # Save the best model based on validation accuracy
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'best_model.pth')

In [16]:
train_and_val_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=10)

Training - Epoch 1/10, Loss: 2.2903, Accuracy: 0.4292
Validation Accuracy: 0.8711
Training - Epoch 2/10, Loss: 0.9901, Accuracy: 0.7177
Validation Accuracy: 0.8774
Training - Epoch 3/10, Loss: 0.7434, Accuracy: 0.7703
Validation Accuracy: 0.9050
Training - Epoch 4/10, Loss: 0.6308, Accuracy: 0.8043
Validation Accuracy: 0.9069
Training - Epoch 5/10, Loss: 0.5644, Accuracy: 0.8271
Validation Accuracy: 0.9088
Training - Epoch 6/10, Loss: 0.5133, Accuracy: 0.8376
Validation Accuracy: 0.9050
Training - Epoch 7/10, Loss: 0.4732, Accuracy: 0.8463
Validation Accuracy: 0.9088
Training - Epoch 8/10, Loss: 0.4468, Accuracy: 0.8513
Validation Accuracy: 0.9220
Training - Epoch 9/10, Loss: 0.4245, Accuracy: 0.8611
Validation Accuracy: 0.9151
Training - Epoch 10/10, Loss: 0.4086, Accuracy: 0.8638
Validation Accuracy: 0.9132


In [17]:
# for param in model.base_model.features[:6].parameters():
#     param.requires_grad = False

for param in model.base_model.features[:2].parameters():
    param.requires_grad = False
    
# Re-define optimizer for fine-tuning
optimizer = optim.SGD(model.parameters(), lr=0.001)

# Fine-tune the model
train_and_val_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=15)

Training - Epoch 1/15, Loss: 0.3933, Accuracy: 0.8685
Validation Accuracy: 0.9182
Training - Epoch 2/15, Loss: 0.3630, Accuracy: 0.8829
Validation Accuracy: 0.9189
Training - Epoch 3/15, Loss: 0.3380, Accuracy: 0.8889
Validation Accuracy: 0.9220
Training - Epoch 4/15, Loss: 0.3302, Accuracy: 0.8992
Validation Accuracy: 0.9208
Training - Epoch 5/15, Loss: 0.3412, Accuracy: 0.8921
Validation Accuracy: 0.9208
Training - Epoch 6/15, Loss: 0.3131, Accuracy: 0.8985
Validation Accuracy: 0.9214
Training - Epoch 7/15, Loss: 0.3100, Accuracy: 0.8985
Validation Accuracy: 0.9226
Training - Epoch 8/15, Loss: 0.3110, Accuracy: 0.8993
Validation Accuracy: 0.9189
Training - Epoch 9/15, Loss: 0.3183, Accuracy: 0.8984
Validation Accuracy: 0.9214
Training - Epoch 10/15, Loss: 0.3089, Accuracy: 0.8984
Validation Accuracy: 0.9220
Training - Epoch 11/15, Loss: 0.3176, Accuracy: 0.8990
Validation Accuracy: 0.9164
Training - Epoch 12/15, Loss: 0.3183, Accuracy: 0.8957
Validation Accuracy: 0.9233
Training - Ep

In [18]:
def evaluate_model_on_test_set(model, test_loader, device, class_mapping):
    model.eval()
    predictions = []
    ids = []
    
    with torch.no_grad():
        for images, image_ids in test_loader:
            images = images.to(device)
            outputs = model(images)
            _, predicted_classes = torch.max(outputs, 1)
            
            predicted_classes = [class_mapping[p.item()] for p in predicted_classes]
            
        
            predictions.extend(predicted_classes)
            ids.extend(image_ids)  # image_ids chính là các ID từ file test.csv
    
    return ids, predictions

class_mapping = {i: f'class_{i+1}' for i in range(71)}

In [19]:
ids, predictions = evaluate_model_on_test_set(model, test_loader, device, class_mapping)
renamed_ids = list(range(len(ids)))
results = pd.DataFrame({
    'ID': renamed_ids, 
    'TARGET': predictions 
})

results.to_csv('submission11.csv', index=False)

# Densenet201

# Ver 8

In [27]:
train_size = int(0.8 * len(train_dataset))
valid_size = len(train_dataset) - train_size
trainset, validset = random_split(train_dataset, [train_size, valid_size])

batch_size = 64
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(validset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [28]:
class CustomDenseNet201(nn.Module):
    def __init__(self, num_classes=71):
        super(CustomDenseNet201, self).__init__()
        
        # Load pre-trained DenseNet201 model
        self.base_model = models.densenet201(pretrained=True)
        
        # Freeze earlier layers (feature extraction layers)
        for param in self.base_model.features.parameters():
            param.requires_grad = False  # Freeze all feature extraction layers

        # Replace the classifier layer with a new fully connected layer
        self.base_model.classifier = nn.Sequential(
            nn.Linear(self.base_model.classifier.in_features, 256),  # in_features based on DenseNet architecture
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)  # Final output layer with num_classes outputs
        )

    def forward(self, x):
        return self.base_model(x)

In [29]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CustomDenseNet201(num_classes=71).to(device)
optimizer = optim.RMSprop(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [30]:
def train_and_val_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=10):
    best_val_acc = 0.0
    for epoch in range(num_epochs):
        model.train()  # Set to training mode
        running_loss, running_corrects = 0.0, 0

        # Training loop
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            _, preds = torch.max(outputs, 1)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = running_corrects.double() / len(train_loader.dataset)
        print(f'Training - Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}')

        # Validation loop
        model.eval()  # Set to evaluation mode
        val_corrects = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                val_corrects += torch.sum(preds == labels.data)

        val_acc = val_corrects.double() / len(val_loader.dataset)
        print(f'Validation Accuracy: {val_acc:.4f}')

        # Save the best model based on validation accuracy
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'best_model.pth')

In [31]:
train_and_val_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=10)

Training - Epoch 1/10, Loss: 2.3120, Accuracy: 0.4309
Validation Accuracy: 0.8409
Training - Epoch 2/10, Loss: 0.9820, Accuracy: 0.7280
Validation Accuracy: 0.8730
Training - Epoch 3/10, Loss: 0.7057, Accuracy: 0.7937
Validation Accuracy: 0.8943
Training - Epoch 4/10, Loss: 0.5914, Accuracy: 0.8244
Validation Accuracy: 0.8987
Training - Epoch 5/10, Loss: 0.5094, Accuracy: 0.8441
Validation Accuracy: 0.9069
Training - Epoch 6/10, Loss: 0.4564, Accuracy: 0.8615
Validation Accuracy: 0.8962
Training - Epoch 7/10, Loss: 0.4188, Accuracy: 0.8688
Validation Accuracy: 0.9050
Training - Epoch 8/10, Loss: 0.3817, Accuracy: 0.8784
Validation Accuracy: 0.8956
Training - Epoch 9/10, Loss: 0.3542, Accuracy: 0.8888
Validation Accuracy: 0.9044
Training - Epoch 10/10, Loss: 0.3197, Accuracy: 0.8957
Validation Accuracy: 0.9132


In [32]:
for param in model.base_model.features[:2].parameters():
    param.requires_grad = False
    
# Re-define optimizer for fine-tuning
optimizer = optim.SGD(model.parameters(), lr=0.001)

# Fine-tune the model
train_and_val_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=15)

Training - Epoch 1/15, Loss: 0.3367, Accuracy: 0.8910
Validation Accuracy: 0.9151
Training - Epoch 2/15, Loss: 0.3045, Accuracy: 0.9031
Validation Accuracy: 0.9220
Training - Epoch 3/15, Loss: 0.3008, Accuracy: 0.9073
Validation Accuracy: 0.9233
Training - Epoch 4/15, Loss: 0.2943, Accuracy: 0.9095
Validation Accuracy: 0.9220
Training - Epoch 5/15, Loss: 0.3033, Accuracy: 0.9042
Validation Accuracy: 0.9233
Training - Epoch 6/15, Loss: 0.2960, Accuracy: 0.9091
Validation Accuracy: 0.9220
Training - Epoch 7/15, Loss: 0.2816, Accuracy: 0.9146
Validation Accuracy: 0.9220
Training - Epoch 8/15, Loss: 0.2785, Accuracy: 0.9122
Validation Accuracy: 0.9201
Training - Epoch 9/15, Loss: 0.2690, Accuracy: 0.9143
Validation Accuracy: 0.9220
Training - Epoch 10/15, Loss: 0.2721, Accuracy: 0.9177
Validation Accuracy: 0.9182
Training - Epoch 11/15, Loss: 0.2735, Accuracy: 0.9157
Validation Accuracy: 0.9189
Training - Epoch 12/15, Loss: 0.2751, Accuracy: 0.9150
Validation Accuracy: 0.9176
Training - Ep

In [33]:
def evaluate_model_on_test_set(model, test_loader, device, class_mapping):
    model.eval()
    predictions = []
    ids = []
    
    with torch.no_grad():
        for images, image_ids in test_loader:
            images = images.to(device)
            outputs = model(images)
            _, predicted_classes = torch.max(outputs, 1)
            
            predicted_classes = [class_mapping[p.item()] for p in predicted_classes]
            
        
            predictions.extend(predicted_classes)
            ids.extend(image_ids)  # image_ids chính là các ID từ file test.csv
    
    return ids, predictions

class_mapping = {i: f'class_{i+1}' for i in range(71)}

In [34]:
ids, predictions = evaluate_model_on_test_set(model, test_loader, device, class_mapping)
renamed_ids = list(range(len(ids)))
results = pd.DataFrame({
    'ID': renamed_ids, 
    'TARGET': predictions 
})

results.to_csv('submission12.csv', index=False)

# VER 8.1

In [4]:
train_size = int(0.8 * len(train_dataset))
valid_size = len(train_dataset) - train_size
trainset, validset = random_split(train_dataset, [train_size, valid_size])

batch_size = 64
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(validset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [5]:
class CustomDenseNet201(nn.Module):
    def __init__(self, num_classes=71):
        super(CustomDenseNet201, self).__init__()
        
        # Load pre-trained DenseNet201 model
        self.base_model = models.densenet201(pretrained=True)
        
        # Freeze earlier layers (feature extraction layers)
        for param in self.base_model.features.parameters():
            param.requires_grad = False  # Freeze all feature extraction layers

        # Replace the classifier layer with a new fully connected layer
        self.base_model.classifier = nn.Sequential(
            nn.Linear(self.base_model.classifier.in_features, 256),  # in_features based on DenseNet architecture
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)  # Final output layer with num_classes outputs
        )

    def forward(self, x):
        return self.base_model(x)

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CustomDenseNet201(num_classes=71).to(device)
optimizer = optim.RMSprop(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)
criterion = nn.CrossEntropyLoss()

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet201_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet201_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet201-c1103571.pth" to /root/.cache/torch/hub/checkpoints/densenet201-c1103571.pth
100%|██████████| 77.4M/77.4M [00:00<00:00, 192MB/s]


In [7]:
def train_and_val_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=10):
    best_val_acc = 0.0
    for epoch in range(num_epochs):
        model.train()  # Set to training mode
        running_loss, running_corrects = 0.0, 0

        # Training loop
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            _, preds = torch.max(outputs, 1)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = running_corrects.double() / len(train_loader.dataset)
        print(f'Training - Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}')

        # Validation loop
        model.eval()  # Set to evaluation mode
        val_corrects = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                val_corrects += torch.sum(preds == labels.data)

        val_acc = val_corrects.double() / len(val_loader.dataset)
        print(f'Validation Accuracy: {val_acc:.4f}')

        # Save the best model based on validation accuracy
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'best_model.pth')

In [ ]:
train_and_val_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=10)

Training - Epoch 1/10, Loss: 2.4028, Accuracy: 0.4097
Validation Accuracy: 0.7994


KeyboardInterrupt: 

In [9]:
for param in model.base_model.features[:0].parameters():
    param.requires_grad = False
    
# Re-define optimizer for fine-tuning
optimizer = optim.SGD(model.parameters(), lr=0.001)

# Fine-tune the model
train_and_val_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=10)

Training - Epoch 1/10, Loss: 0.1999, Accuracy: 0.9342
Validation Accuracy: 0.9352
Training - Epoch 2/10, Loss: 0.1737, Accuracy: 0.9449
Validation Accuracy: 0.9377
Training - Epoch 3/10, Loss: 0.1644, Accuracy: 0.9427
Validation Accuracy: 0.9377
Training - Epoch 4/10, Loss: 0.1613, Accuracy: 0.9520
Validation Accuracy: 0.9371
Training - Epoch 5/10, Loss: 0.1581, Accuracy: 0.9470
Validation Accuracy: 0.9346
Training - Epoch 6/10, Loss: 0.1570, Accuracy: 0.9508
Validation Accuracy: 0.9390
Training - Epoch 7/10, Loss: 0.1591, Accuracy: 0.9498
Validation Accuracy: 0.9352
Training - Epoch 8/10, Loss: 0.1496, Accuracy: 0.9512
Validation Accuracy: 0.9365
Training - Epoch 9/10, Loss: 0.1504, Accuracy: 0.9542
Validation Accuracy: 0.9390
Training - Epoch 10/10, Loss: 0.1466, Accuracy: 0.9520
Validation Accuracy: 0.9340


In [10]:
def evaluate_model_on_test_set(model, test_loader, device, class_mapping):
    model.eval()
    predictions = []
    ids = []
    
    with torch.no_grad():
        for images, image_ids in test_loader:
            images = images.to(device)
            outputs = model(images)
            _, predicted_classes = torch.max(outputs, 1)
            
            predicted_classes = [class_mapping[p.item()] for p in predicted_classes]
            
        
            predictions.extend(predicted_classes)
            ids.extend(image_ids)  # image_ids chính là các ID từ file test.csv
    
    return ids, predictions

class_mapping = {i: f'class_{i+1}' for i in range(71)}

In [11]:
ids, predictions = evaluate_model_on_test_set(model, test_loader, device, class_mapping)
renamed_ids = list(range(len(ids)))
results = pd.DataFrame({
    'ID': renamed_ids, 
    'TARGET': predictions 
})

results.to_csv('submission18.csv', index=False)

# Esemble

In [14]:
train_size = int(0.8 * len(train_dataset))
valid_size = len(train_dataset) - train_size

trainset, validset = random_split(train_dataset, [train_size, valid_size])

# Create DataLoaders
batch_size = 64
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(validset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [15]:
class CustomDenseNet201(nn.Module):
    def __init__(self, num_classes=71):
        super(CustomDenseNet201, self).__init__()
        self.base_model = models.densenet201(pretrained=True)
        
        # Freeze earlier layers (optional, based on your strategy)
        for param in self.base_model.features.parameters():
            param.requires_grad = False
        
        # Modify the classifier
        in_features = self.base_model.classifier.in_features
        self.base_model.classifier = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )
    
    def forward(self, x):
        return self.base_model(x)

class CustomResNet50(nn.Module):
    def __init__(self, num_classes=71):
        super(CustomResNet50, self).__init__()
        self.base_model = models.resnet50(pretrained=True)
        
        # Freeze earlier layers (optional)
        for param in self.base_model.parameters():
            param.requires_grad = False
        
        # Modify the classifier
        in_features = self.base_model.fc.in_features
        self.base_model.fc = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )
    
    def forward(self, x):
        return self.base_model(x)

In [16]:
class EnsembleModel(nn.Module):
    def __init__(self, num_classes=71):
        super(EnsembleModel, self).__init__()
        self.densenet = CustomDenseNet201(num_classes)
        self.resnet = CustomResNet50(num_classes)
        
        # Combine outputs
        self.fc = nn.Linear(num_classes * 2, num_classes)  # Combine predictions from both models
    
    def forward(self, x):
        densenet_output = self.densenet(x)
        resnet_output = self.resnet(x)
        
        # Concatenate outputs
        combined_output = torch.cat((densenet_output, resnet_output), dim=1)
        
        # Pass through final classifier
        final_output = self.fc(combined_output)
        
        return final_output

In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = EnsembleModel(num_classes=71).to(device)

optimizer = torch.optim.RMSprop(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [18]:
def train_and_val_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=10):
    best_val_acc = 0.0
    for epoch in range(num_epochs):
        model.train()  # Set to training mode
        running_loss, running_corrects = 0.0, 0

        # Training loop
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            _, preds = torch.max(outputs, 1)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = running_corrects.double() / len(train_loader.dataset)
        print(f'Training - Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}')

        # Validation loop
        model.eval()  # Set to evaluation mode
        val_corrects = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                val_corrects += torch.sum(preds == labels.data)

        val_acc = val_corrects.double() / len(val_loader.dataset)
        print(f'Validation Accuracy: {val_acc:.4f}')

        # Save the best model based on validation accuracy
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'best_model.pth')

In [19]:
train_and_val_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=15)

Training - Epoch 1/15, Loss: 2.4865, Accuracy: 0.4456
Validation Accuracy: 0.7717
Training - Epoch 2/15, Loss: 0.7420, Accuracy: 0.7865
Validation Accuracy: 0.8453
Training - Epoch 3/15, Loss: 0.5494, Accuracy: 0.8295
Validation Accuracy: 0.8302
Training - Epoch 4/15, Loss: 0.4692, Accuracy: 0.8491
Validation Accuracy: 0.8987
Training - Epoch 5/15, Loss: 0.3985, Accuracy: 0.8727
Validation Accuracy: 0.8855
Training - Epoch 6/15, Loss: 0.3613, Accuracy: 0.8787
Validation Accuracy: 0.9013
Training - Epoch 7/15, Loss: 0.3348, Accuracy: 0.8933
Validation Accuracy: 0.9000
Training - Epoch 8/15, Loss: 0.3076, Accuracy: 0.8988
Validation Accuracy: 0.8931
Training - Epoch 9/15, Loss: 0.2839, Accuracy: 0.9039
Validation Accuracy: 0.9025
Training - Epoch 10/15, Loss: 0.2721, Accuracy: 0.9083
Validation Accuracy: 0.8931
Training - Epoch 11/15, Loss: 0.2517, Accuracy: 0.9179
Validation Accuracy: 0.8950
Training - Epoch 12/15, Loss: 0.2382, Accuracy: 0.9198
Validation Accuracy: 0.8673
Training - Ep

In [20]:
for param in model.densenet.base_model.features[-2:].parameters():  # Unfreeze last 2 layers of DenseNet201
    param.requires_grad = True

for param in model.resnet.base_model.layer4.parameters():  # Unfreeze layer 4 of ResNet50
    param.requires_grad = True

# Re-define optimizer for fine-tuning
optimizer = torch.optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=0.0001)
train_and_val_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=10)

Training - Epoch 1/10, Loss: 0.3837, Accuracy: 0.8767
Validation Accuracy: 0.8818
Training - Epoch 2/10, Loss: 0.3500, Accuracy: 0.8881
Validation Accuracy: 0.8780
Training - Epoch 3/10, Loss: 0.3370, Accuracy: 0.8903
Validation Accuracy: 0.8899
Training - Epoch 4/10, Loss: 0.3245, Accuracy: 0.8944
Validation Accuracy: 0.8937
Training - Epoch 5/10, Loss: 0.3146, Accuracy: 0.8995
Validation Accuracy: 0.8950
Training - Epoch 6/10, Loss: 0.2776, Accuracy: 0.9073
Validation Accuracy: 0.8994
Training - Epoch 7/10, Loss: 0.2821, Accuracy: 0.9051
Validation Accuracy: 0.9057
Training - Epoch 8/10, Loss: 0.2754, Accuracy: 0.9108
Validation Accuracy: 0.9044
Training - Epoch 9/10, Loss: 0.2569, Accuracy: 0.9163
Validation Accuracy: 0.9038
Training - Epoch 10/10, Loss: 0.2500, Accuracy: 0.9176
Validation Accuracy: 0.9031


In [21]:
def evaluate_model_on_test_set(model, test_loader, device, class_mapping):
    model.eval()
    predictions = []
    ids = []
    
    with torch.no_grad():
        for images, image_ids in test_loader:
            images = images.to(device)
            outputs = model(images)
            _, predicted_classes = torch.max(outputs, 1)
            
            predicted_classes = [class_mapping[p.item()] for p in predicted_classes]
            
        
            predictions.extend(predicted_classes)
            ids.extend(image_ids)  # image_ids chính là các ID từ file test.csv
    
    return ids, predictions

class_mapping = {i: f'class_{i+1}' for i in range(71)}

In [22]:
ids, predictions = evaluate_model_on_test_set(model, test_loader, device, class_mapping)
renamed_ids = list(range(len(ids)))
results = pd.DataFrame({
    'ID': renamed_ids, 
    'TARGET': predictions 
})

results.to_csv('submission16.csv', index=False)